In [65]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [66]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [67]:

from pandas.io import sql as psql

In [68]:
from lib.connected_db import PgsqlExecutor

In [69]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [70]:

test_connect = PgsqlExecutor(research_db)

In [71]:
db = PgsqlExecutor(snpsource)

In [72]:
item_dict = {}

In [73]:
sql = """

select tradingitemid, tickersymbol from v2mv_sbt_company where  
is_primary_exchange = true and exchangesymbol not ilike 'mutualfund'
    and securityname ilike 'common%stock' 
             and exchangecountry = 'USA'
"""

data = db.fetch_all_rows(sql)
for row in data.query_data:
    item_dict[row.tickersymbol.upper()] = row

In [74]:
def get_current_price(symbol):
    sql = """
    SELECT  priceclose
	    FROM public.v_price_timeseries 
	        WHERE tradingitemid = %s 
	        AND pricingdate >= now()::Date -3
            
            order by pricingdate desc limit 1
    
    """
    rv = db.fetch_one_row(sql, args=[item_dict[symbol.upper()].tradingitemid])
    if rv.row_count > 0:
        return rv.query_data.priceclose

In [75]:
get_current_price('aapl')

338.8

In [12]:
import csv

In [13]:
data = []
with open('merton_model_V1.3_corrected_rfrate.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        data.append(row)

In [14]:
data[0:2]

[['',
  'tickersymbol',
  'exchangesymbol',
  'dd',
  'pd',
  'S&P Rating',
  'Rating Date',
  'Last Review Date',
  'Regulatory Identifiers',
  'CreditWatch/ Outlook',
  '',
  "Moody's Rating",
  'Rating Action',
  'Date'],
 ['956',
  'CHK',
  'NYSE',
  '-20.40143789',
  '1',
  'CC',
  '1-May-20',
  '1-May-20',
  'EE',
  'Negative',
  '',
  'Ca',
  'Downgrade',
  '24-Apr-20']]

In [15]:
columns = ["id", "symbol", "dd", "pb", "snp_rating", "CreditWatch", "dcf_predict"]

In [16]:
merton_model = {}

for row in data[1:]:
    
    symbol = row[1]
    
    value= {
    "id":row[0],
    "symbol":row[1],
    "dd":row[3],
    "pd":row[4],
        "snp_rating":row[5],
        "credit_watch":row[9],
        "dcf_predict":None
      
    }
    merton_model[symbol] = value

In [17]:
import requests

In [18]:
url = "https://api.nasdaq.com/api/calendar/earnings?date=2020-06-09"

In [19]:
response = requests.get(url)
reponse_values = response.json()['data']['rows']

In [20]:
for row in reponse_values:
    print (row)
    print(row["symbol"])

{'eps': '$0.27', 'surprise': '-3.57', 'time': 'time-not-supplied', 'symbol': 'BF.B', 'name': 'Brown Forman Corporation', 'marketCap': '$31,365,995,676', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '$0.28', 'noOfEsts': '6'}
BF.B
{'eps': '$0.27', 'surprise': '999', 'time': 'time-not-supplied', 'symbol': 'BF.A', 'name': 'Brown Forman Corporation', 'marketCap': '$27,942,512,002', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '', 'noOfEsts': '0'}
BF.A
{'eps': '($0.12)', 'surprise': '-9.09', 'time': 'time-not-supplied', 'symbol': 'CHWY', 'name': 'Chewy, Inc.', 'marketCap': '$19,748,882,700', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '($0.11)', 'noOfEsts': '6'}
CHWY
{'eps': '($0.53)', 'surprise': '-51.43', 'time': 'time-not-supplied', 'symbol': 'TIF', 'name': 'Tiffany & Co.', 'marketCap': '$14,497,287,142', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '($0.35)', 'noOfEsts': '3'}
TIF
{'eps': '($0.91)', 'surprise': '-160', 'time': 'time-not-supplied', 'symbol': 'FIVE', 'n

In [77]:
from mapper.predict_logic_mapper import MapperPredictLogic, simplejson

In [78]:
mapper_logic = MapperPredictLogic(test_connect)

In [79]:
data = mapper_logic.get_dcf_data("bkh")

print(data, list(data))

[-1.462, -470.2, 0.0, -100.0, -698.6758339699517] [-1.462, -470.2, 0.0, -100.0, -698.6758339699517]


In [80]:
dto = mapper_logic.get_active_session('dcf')
dto.features

['last_growth_rate',
 'last_market_dcf',
 'last_dcf',
 'dcf_to_price',
 'market_dcf_to_price']

In [81]:
dto = mapper_logic.get_dto(name='dcf')
dto = mapper_logic.select(dto, where_column='name')

In [82]:
import pandas as pd

csv_file = "make_dcf_model_0612_put_83_accuracy.csv"
data = pd.read_csv(csv_file, header=0, index_col=0)

In [83]:
data.head()

symbol  current_price  last_price  last_growth_rate  current_growth_rate  \
4      A          24.10       29.24            -2.106               -0.897   
5      A          33.29       24.10            -0.897                1.576   
6      A          34.85       33.29             1.576                1.925   
7      A          36.74       34.85             1.925                1.718   
8      A          15.63       36.74             1.718                1.874   

   market_dcf    dcf  growth_changes  market_dcf_change  dcf_change  \
4        23.9    0.1           1.209         -20.519836    0.341997   
5        31.4  165.1           2.473          31.120332  684.647303   
6        38.6  338.1           0.349          21.628117  519.675578   
7        44.7  320.6          -0.207          17.503587  -50.215208   
8        26.2  431.9           0.156         -50.353838  302.939575   

   last_fiscal_year  last_market_dcf  last_dcf  dcf_to_price  \
4            2003.0             29.9       0.0   -100.000000   
5            2004.0             23.9       0.1    -99.585062   
6            2005.0             31.4     165.1    395.944728   
7            2006.0             38.6     338.1    870.157819   
8            2007.0             44.7     320.6    772.618400   

   market_dcf_to_price  price_change  direction  price_change_perf  
4             2.257182         -5.14         -1         -17.578659  
5            -0.829876          9.19          1          38.132780  
6            -5.677381          1.56          1           4.686092  
7            10.760402          1.89          1           5.423242  
8            21.665759        -21.11         -1         -57.457812

In [84]:
def convert_direction_up(value):
    if value > 25:
        return 1
    
    return -1
def convert_direction_down(value):
    if value < -25:
        return -1
    
    return 1

data["direction_up"] = data["price_change"].apply(convert_direction_up)
data["direction_down"] = data["price_change"].apply(convert_direction_down)

In [85]:
from numpy import loadtxt
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [86]:
features = [ "last_growth_rate",
                         "last_market_dcf", "last_dcf", 
                           "dcf_to_price", "market_dcf_to_price"]

X = data[features].values
Y = (data.loc[:, 'price_change_perf'].values) 
y_up = (data.loc[:, 'direction_up'].values) 
y_down = (data.loc[:, 'direction_down'].values) 

In [87]:
new_data = data[
    (data["last_growth_rate"] < 100)
    & (data["last_growth_rate"] > -100)
    & (data["last_market_dcf"] > -1000)
    & (data["last_market_dcf"] < 1000)
    & (data["last_dcf"] < 2000)
    & (data["last_dcf"] > -2000)
    & (data["dcf_to_price"] > -2000)
    & (data["dcf_to_price"] < 2000)
    & (data["market_dcf_to_price"] > -2000)
    & (data["market_dcf_to_price"] < 2000)
   
    
    
]

In [88]:
X = new_data[features].values
Y = (new_data.loc[:, 'price_change_perf'].values) 
y_up = (new_data.loc[:, 'direction_up'].values) 
y_down = (new_data.loc[:, 'direction_down'].values) 



In [89]:
dto = mapper_logic.get_active_session('dcf_binary_up')

In [90]:
pred_up = dto.logic.predict(X)

In [91]:
accuracy = accuracy_score(y_up==1, pred_up==1)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 97.34%


In [92]:
dto_down = mapper_logic.get_active_session('dcf_binary_down')

In [93]:
pred_down = dto_down.logic.predict(X)

In [94]:
test_accuracy(pred_down, y_down, -1)

0.9087301587301587 21878


In [95]:
dto1 = mapper_logic.get_active_session('dcf')

In [96]:
pred1 = dto1.logic.predict(X)

In [97]:
accuracy = accuracy_score(Y >= 30, pred1 >= 30)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 73.81%


In [98]:
print(len([i for i in Y if i >= 30]), len(Y))

5741 21878


In [99]:
accuracy = accuracy_score(Y < -30, pred1 < -30)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 84.15%


In [100]:
print(len([i for i in Y if i < -30]), len(Y))

3583 21878


In [101]:
dto2 = mapper_logic.get_active_session('dcf_put')

In [102]:
pred2 = dto2.logic.predict(X)

In [103]:
accuracy = accuracy_score(Y >= 30, pred2 >= 30)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 74.07%


In [104]:
accuracy = accuracy_score(Y < -30, pred2 < -30)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 84.26%


In [105]:
accuracy = accuracy_score(pred2 >= 30, Y >= 30)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(pred2 >= 30, Y >= 30)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 74.07%
Accuracy: 74.07%


In [106]:
accuracy = accuracy_score(Y < -40, pred2 < -40)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(Y<-40, pred1 < -40)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.96%
Accuracy: 88.87%


In [52]:
print(len([i for i in Y if i < -40]), len(Y))

2467 21878


In [53]:
count_up = 0
total = 0
for i, j in zip(Y, pred2):
    if j < -40:
        total += 1
        if i < -40:
            count_up += 1
        
print(count_up/total, len([i for i in Y if i < -40]))

0.84 2467


In [54]:
count_up = 0
total = 0
for i, j in zip(Y, pred1):
    if j < -40:
        total += 1
        if i < -40:
            count_up += 1
        
print(count_up/total, len([i for i in Y if i < -40]))

0.6222222222222222 2467


In [55]:
count_up = 0
total = 0
for i, j in zip(Y, pred2):
    if j > 30:
        total += 1
        if i > 30:
            count_up += 1
        
print(count_up/total, len([i for i in Y if i > 30]))

0.84 5739


In [56]:
count_up = 0
total = 0
for i, j in zip(Y, pred1):
    if j > 30:
        total += 1
        if i > 30:
            count_up += 1
        
print(count_up/total, len([i for i in Y if i > 30]))

0.9285714285714286 5739


In [57]:
def test_accuracy(pred, Y, test_value):
    count_up = 0
    total = 0
    if test_value > 0:
        for i, j in zip(Y, pred):
            if j >= test_value:
                total += 1
                if i >= test_value:
                    count_up += 1
    if test_value < 0:
        for i, j in zip(Y, pred):
            if j <= test_value:
                total += 1
                if i <= test_value:
                    count_up += 1
        

    print(count_up/total, len(pred))

In [58]:
test_accuracy(pred2, Y, 30)

0.84 21878


In [59]:
test_accuracy(pred1, Y, 30)

0.9285714285714286 21878


In [60]:
test_accuracy(pred_up, y_up, 1)

0.8936170212765957 21878


In [61]:
test_accuracy(pred2, Y, -40)

0.84 21878


In [62]:
test_accuracy(pred1, Y, -40)

0.6222222222222222 21878


In [63]:
test_accuracy(pred_down, y_down, -1)

0.9087301587301587 21878


In [ ]:
dto = 

In [328]:
def convert_direction(value):
    if value < -30:
        return -1
    
    return 1

data["direction"] = data["price_change"].apply(convert_direction)

In [329]:
data["last_growth_rate"].describe()

count    32086.000000
mean        -1.470009
std        120.385685
min     -12963.325000
25%         -1.212000
50%          0.029000
75%          0.533000
max       3279.901000
Name: last_growth_rate, dtype: float64

In [330]:
data["last_market_dcf"].describe()

count    3.101800e+04
mean    -2.243384e+12
std      3.951048e+14
min     -6.958557e+16
25%      1.900000e+00
50%      2.400000e+01
75%      6.400000e+01
max      3.072990e+11
Name: last_market_dcf, dtype: float64

In [331]:
data["last_dcf"].describe()

count    3.101800e+04
mean     7.537327e+13
std      1.316980e+16
min     -3.536182e+16
25%      0.000000e+00
50%      1.200000e+00
75%      7.730000e+01
max      2.316352e+18
Name: last_dcf, dtype: float64

In [332]:
data["dcf_to_price"].describe()

count    3.101700e+04
mean              NaN
std               NaN
min              -inf
25%     -1.000000e+02
50%     -8.674242e+01
75%      1.409091e+02
max               inf
Name: dcf_to_price, dtype: float64

In [333]:
data.shape

(32396, 18)

In [334]:
new_data = data[
    (data["last_growth_rate"] < 100)
    & (data["last_growth_rate"] > -100)
    & (data["last_market_dcf"] > -1000)
    & (data["last_market_dcf"] < 1000)
    & (data["last_dcf"] < 2000)
    & (data["last_dcf"] > -2000)
    & (data["dcf_to_price"] > -2000)
    & (data["dcf_to_price"] < 2000)
    & (data["market_dcf_to_price"] > -2000)
    & (data["market_dcf_to_price"] < 2000)
    
    
    
]

In [335]:
features = [ "last_growth_rate",
                         "last_market_dcf", "last_dcf", 
                           "dcf_to_price", "market_dcf_to_price"]
X = new_data[features].values
Y = new_data['direction'].values

In [336]:
new_data.shape

(21878, 18)

In [337]:
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
def calculate_score(Ytest, pred):
    total_consideration = 0
    correct_count = 0
    for i, j in zip(Ytest, pred):
        if i in(1, -1):
            total_consideration += 1
            if i==j:
                correct_count += 1
    return float(correct_count) / total_consideration

indices = np.arange(len(Y))
score = 0.9585
score_process = 0.0
num_processes = 0
while score_process < score and num_processes < 50:
    num_processes += 1
    clf = ExtraTreesClassifier()
    generator = np.random.RandomState(50)
    generator.shuffle(indices)
    X = X[indices]
    Y = Y[indices]
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.35)
    clf.fit(Xtrain, Ytrain)
    print ("Training score:\n")
    print (clf.score(X, Y))

    pred = clf.predict(Xtest)
    score_process = calculate_score(Ytest, pred)
    print (score_process)
    

Training score:

0.9898985282018467
0.9711412901540872


In [338]:
import pickle

In [444]:

test_connect = PgsqlExecutor(research_db)


In [340]:
from mapper.predict_logic_mapper import MapperPredictLogic, simplejson

In [341]:
logic_mapper = MapperPredictLogic(test_connect)

In [342]:


dto = logic_mapper.get_dto(
name="dcf_binary_down",
features=simplejson.dumps(features),
logic=pickle.dumps(clf)
    
)

In [343]:
logic_mapper.insert(dto, update_on_conflict="name")

insert ExecutionResults(query_data=Record(id=6, name='dcf_binary_down', features=['last_growth_rate', 'last_market_dcf', 'last_dcf', 'dcf_to_price', 'market_dcf_to_price'], logic=<memory at 0x14bdb2108>, created=datetime.datetime(2020, 6, 12, 22, 10, 4, 636830)), row_count=1)


< DtoPredictLogic, {'id': 6, 'name': 'dcf_binary_down', 'features': ['last_growth_rate', 'last_market_dcf', 'last_dcf', 'dcf_to_price', 'market_dcf_to_price'], 'logic': <memory at 0x14bdb2108>, 'created': datetime.datetime(2020, 6, 12, 22, 10, 4, 636830)} >